<a href="https://colab.research.google.com/github/andtab/absorption-ratio-trading-strategy/blob/main/absorp_ratio_autoencoder_vs_pca_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predicting Systemic Shock via Absorption Ratio Using Decomposition by PCA and AE

Using PCA and autoencoder decomposition methods to predict system shock via Absorption Ratio. Based on the following paper linked below: <br>
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.466.5471&rep=rep1&type=pdf

Accompanying presentation can be found here:
https://docs.google.com/presentation/d/1VEUksJE7PvS3WN7hw2w7jZnvFYBALHFWJ8kCWMqxkas/edit#slide=id.p20

In [ ]:
import pandas as pd
import numpy as np
import sklearn.decomposition
import os

%matplotlib inline
import matplotlib.pyplot as plt
import sklearn.model_selection


# Install factor analyzer on Google Colab
!pip install factor_analyzer
from factor_analyzer.factor_analyzer import calculate_kmo

# AT - Additional imports
from sklearn.decomposition import PCA
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import regularizers
import time

In [ ]:
#1. Go to WRDS CRSP database and obtain the price and ticker data for the following industry ETFs:
#    IYW IYJ IBB IGV IXP IYT VNQ PBS PBJ KCE KIE KBE XSD ITB ITA IHI IHF IHE IEZ FDN
#2. set the starting and ending dates as follows: 1/3/2007 and 12/31/2019

#3. Unstack the prices and save the unstacked price data into a csv file called:
#    20industries_ETF_prc_UNSTACKED.csv

#UNSTACK PRICES

StackedPrices = pd.read_csv('20industries_ETF_prc_STACKED.csv')
StackedPrices.date = pd.to_datetime(StackedPrices.date)
StackedPrices.PRC = StackedPrices.PRC.abs()
StackedPrices.drop(['PERMNO'], axis=1, inplace=True)
UnstackedPrices = pd.pivot_table(StackedPrices, values = 'PRC', index = 'date', columns = 'TICKER')
UnstackedPrices.sort_index(inplace=True)
print(UnstackedPrices.shape)
UnstackedPrices.head(5)
UnstackedPrices.to_csv('20industries_ETF_prc_UNSTACKED.csv')

In [ ]:

# load dataset

asset_prices = pd.read_csv('20industries_ETF_prc_UNSTACKED.csv',
                     date_parser=lambda dt: pd.to_datetime(dt, format='%Y-%m-%d'),
                     index_col = 0).dropna()
n_stocks_show = 20
print('Asset prices shape', asset_prices.shape)
asset_prices.iloc[:, :n_stocks_show].head()

### Calculate daily returns

In [ ]:
asset_returns = asset_prices.pct_change(periods=1)
asset_returns = asset_returns.iloc[1:, :]
asset_returns.iloc[:, :n_stocks_show].head()

In [ ]:

def normalize_returns(r_df):
    """
    Normalize, i.e. center and divide by standard deviation raw asset returns data

    Arguments:
    r_df -- a pandas.DataFrame of asset returns

    Return:
    normed_df -- normalized returns
    """

    mean_r = r_df.mean()
    sd_r = r_df.std()
    normed_df = (r_df - mean_r)/sd_r
    

    return normed_df

In [ ]:
normed_r = normalize_returns(asset_returns)
normed_r.iloc[:, :n_stocks_show].head()

#### Part 1 (Implement an exponentially-weighting function to mimic pandas.EWM function)
Define sequence of $X_j$ where $j \subset [0, N]$, an integer taking all values in the interval from 0 to N  $$ X_j =  e^{-\frac{log(2)}{H} \times  \space j}$$
where H is half-life which determines the speed of decay, and $log$ is natural log function.
Then a sequence of exponentially decaying weights $w_j$ is defined as: $$ w_j = \frac{X_j}{ \sum\limits_{i=0}^j X_i } $$

In [ ]:
#Exponent weighting function, to take the place of Pandas EWM function (we will compare their effect)
def exponent_weighting(n_periods, half_life = 252):
    """
    Calculate exponentially smoothed normalized (in probability density function sense) weights

    Arguments:
    n_periods -- number of periods, an integer, N in the formula above
    half_life -- half-life, which determines the speed of decay, h in the formula
    
    Return:
    exp_probs -- exponentially smoothed weights, np.array
    """
    
    #exp_probs = np.zeros(n_periods) # do your own calculation instead of dummy zero array


    arr= np.array(range(0,n_periods))
    num= np.exp(-np.log(2) * arr / half_life)
    denom= np.cumsum(num)
    exp_probs = num / denom
    exp_probs = np.flip(exp_probs)

    
    return exp_probs

In [ ]:
#note the flipping for graphing purposes only, 
#the graph should look like an exponential probability density function
#in reality we want weights applied to returns 
#to decay as returns are older, further in the past
exp_probs = exponent_weighting(252*1)
plt.plot(np.flip(exp_probs), linewidth=3) 
plt.show()

In [ ]:
def absorption_ratio(explained_variance_arr, n_components):
    """
    Calculate absorption ratio via PCA or Autoencoder.  
    
    Arguments:
    explained_variance_arr -- 1D np.array of explained variance by each pricincipal component, in descending order
    n_components -- an integer, a number of principal components to compute absorption ratio
    Formally, the ratio is defined by the variance absorbed by the first n eigenvectors
    The numerator equals the cumulative variance explained by the first n eigenvectors (=n_components) 
    And the denominator equals the total variance
    Return:
    ar -- absorption ratio
    """


    ar = np.cumsum(explained_variance_arr)[n_components - 1]

    return ar

In [ ]:
stock_tickers = asset_returns.columns.values[:]
assert 'SPX' not in stock_tickers, "By accident included SPX index"

half_life = 252             # in (days)
lookback_window = 252 * 1   # in (days)
num_assets = len(stock_tickers)
step_size = 1          # days : 5 - weekly, 21 - monthly, 63 - quarterly

# require 0.8 variance to be explained. How many components are needed?
var_threshold = 0.8     

# fix 20% of principal components for absorption ratio calculation. How much variance (in %) do they explain?
absorb_comp = int((1 / 5) * num_assets)  

print('Half-life = %d' % half_life)
print('Lookback window = %d' % lookback_window)
print('Step size = %d' % step_size)
print('Variance Threshold = ', var_threshold)
print('Number of assets = %d' % num_assets)
print('Number of principal components = %d' % absorb_comp)

In [ ]:
# indexes date on which to compute PCA
days_offset = 4 * 252 
num_days = 6 * 252 + days_offset 
components_ts_index = asset_returns.index[list(range(lookback_window + days_offset, min(num_days, len(asset_returns)), step_size))]

# allocate arrays for storing absorption ratio
components = np.array([np.nan]*len(components_ts_index))
absorp_ratio = np.array([np.nan]*len(components_ts_index))
kmo = np.array([np.nan]*len(components_ts_index))

exp_probs = exponent_weighting(lookback_window, half_life)
assert 'SPX' not in asset_returns.iloc[:lookback_window, :].columns.values, "By accident included SPX index"

In [ ]:
#1. Go to WRDS CRSP database and obtain the Holding Period Return and ticker for the following industry ETFs:
#    VTI AGG
#the holding period return includes dividends, coupon flows etc.
#2. set the starting and ending dates as follows: 1/3/2007 and 12/31/2019
#3. Unstack the holding period returns and save the unstacked returns data into a csv file called:
#    AGG_VTI_UNSTACKED_adjusted_returns.csv

#UNSTACK RETURNS
StackedReturns = pd.read_csv('AGG_VTI_STACKED_adjusted_returns.csv')
StackedReturns.date = pd.to_datetime(StackedReturns.date)
StackedReturns.drop(['PERMNO'], axis=1, inplace=True)
UnstackedReturns = pd.pivot_table(StackedReturns, values = 'RET', index = 'date', columns = 'TICKER')
UnstackedReturns.sort_index(inplace=True)
print(UnstackedReturns.shape)
UnstackedReturns.to_csv('AGG_VTI_UNSTACKED_adjusted_returns.csv')
UnstackedReturns.head(5)


#These are adjusted PERCENT returns of AGG and VTI from WRDS CRSP, including distibutions, coupons, dividends etc.
etf_percent_returns = pd.read_csv('AGG_VTI_UNSTACKED_adjusted_returns.csv')
etf_percent_returns["date"]=pd.to_datetime(etf_percent_returns['date'])
etf_percent_returns.set_index(['date'], inplace=True)
#Use the adjusted PERCENT returns to calculate the etf_log returns:

etf_log_returns = np.log(1 + etf_percent_returns)
#decide which return you are going to use by uncommenting a or b below (only a or b is correct, not both):
#a:omp
#etf_returns = etf_log_returns.copy()
#b:
etf_returns = etf_percent_returns.copy()

In [ ]:

def get_weight(ar_delta):
    '''
    Calculate EQuity / FIncome portfolio weights based on Absorption Ratio delta
    Arguments:
    ar_delta -- Absorption Ratio delta
    
    Return: 
        wgts -- a vector of portfolio weights
    '''


    if ar_delta < -1:
        wgts = [0.5, 0.5]
    elif ar_delta > 1:
        wgts = [0.0, 1.0]
    else:
        wgts = [1.0, 0.0]
    return wgts


In [ ]:

def backtest_strategy(strat_wgts, asset_returns, periods_per_year = 252):
    '''
    Calculate portfolio returns and return portfolio strategy performance
    Arguments:
    
    strat_wgts -- pandas.DataFrame of weights of the assets
    asset_returns -- pandas.DataFrame of asset returns
    periods_per_year -- number of return observations per year
    
    Return: 
        (ann_ret, ann_vol, sharpe) -- a tuple of (annualized return, annualized volatility, sharpe ratio)
    before returning plot the cumulative return of the EWM+AGG portfolio  
    '''

    df_all=pd.merge(strat_wgts,asset_returns, left_index=True, right_index=True, how='inner')

    df_all['EQ_part'] = df_all['EQ'] * df_all['VTI']
    df_all['FI_part'] = df_all['FI'] * df_all['AGG']
    df_all['PORT_ret'] = df_all['EQ_part'] + df_all['FI_part'] 
 
    annualized_return= (1 + df_all['PORT_ret'].mean())**periods_per_year - 1
    annualized_vol = df_all['PORT_ret'].std() * np.sqrt(periods_per_year)
    annualized_sharpe = (df_all['PORT_ret'].mean()/df_all['PORT_ret'].std()) * np.sqrt(periods_per_year)
    #before returning plot the cumulative return of the EWM+AGG portfolio 
    df_all['Cum_PORT_ret'] = (df_all['PORT_ret']).cumsum() + 1

    returns = df_all['Cum_PORT_ret']

    return annualized_return, annualized_vol, annualized_sharpe, returns

In [ ]:
# The goal of this function is to perform dimension reduction on the features
# using either Principal Component Analysis (PCA) (a process for which sklearn 
# has useful functions) or an Autoencoder (AE) 
# (where eigen-decomposition must be built "from scratch")

# The primary inputs of this function will be:
# 1: the dataframe for which decomposition must be performed, and
# 2: the decomposition method selected.

# The outputs of this function will be: 
# 1: the latent features, in order of descending corresponding explained 
# variance, built from eigen-decomposition using either AE or PCA, and
# 2: the similarly ordered explained variance ratios themselves
def decomposition_function(input_dataframe, method = 'AE', \
                           node_count_list = None, max_encode = False):

  # Convert the input dataframe into an array, for guaranteed compatibility
  # with numpy operations
  input_data = input_dataframe.values



  # If selected decomposition method is 'AE', standing for "Autoencoder", the 
  # procedure contained within this block of the IF statement will be performed
  if method == 'AE':
    
    model = Sequential()

    # These are hyperparameters related to the training process of the AE model 
    num_epochs = 20 # Number of passes through the dataset for training purposes

    batch_size = 32 # The number of samples that must be observed before the 
                    # model goes through with an update

    act = 'ReLU' # The type of activation function used with hidden and input layers

    final_act = 'linear' # The type of activation function used just after output layer

    optimizer = keras.optimizers.Adam(learning_rate= 0.0005) # The optimizer used 
                       # to update model performance after 
                       # each epoch. Learning rate can be set here

    loss = 'mean_squared_error' # The loss which the optimizer attempts to 
                                # minimize through each pass of the dataset

    # The immediate input dimensions for the Autoencoder model; this is the 
    # number of features/columns used for prediction input in our dataset
    input_dim = input_data.shape[1]

    # Create the specified number of hidden layers in our network, each having
    # a specified number of nodes 
    for x in range(1, len(node_count_list) + 1):
      model.add(Dense(units = node_count_list[x - 1], activation=act, \
                      input_dim = input_dim, name= 'hidden_layer_' + str(x), \
                    activity_regularizer = regularizers.L1L2(10e-5,10e-5)))

    # Add an output layer to the model
    model.add(Dense(units = input_dim, activation = final_act))
    
    # Compile the model, the final step in generating a model structure that
    # is ready to be fit and trained on a dataset
    model.compile(optimizer = optimizer,
                  loss = loss)

    # Fit the model to the dataset
    history = model.fit(x = input_data, y = input_data,
                    epochs = num_epochs,
                    batch_size = batch_size,
                    shuffle = False,
                    validation_data = (input_data, input_data),
                    verbose = 0)
    
    # When the max_encode parameter is set to 'True', this represents encoding 
    # the features space to the minimum possible dimension based on the 
    # autoencoder architecture. Otherwise, the entire hidden layer architecture
    # is used for decomposition
    input_layer_name = 'hidden_layer_1'

    if max_encode == False:
      output_layer_name = 'hidden_layer_' + str(x)
    
    else:
      
      # Reverse the list of node counts and determine the hidden layer with 
      # the minimum node count
      node_count_list.reverse()
      node_count, idx = min((node_count, idx) for (idx, node_count) in enumerate(node_count_list))
      layer_number = len(node_count_list) - idx
      output_layer_name = 'hidden_layer_' + str(layer_number)


    # Create the intermediate layer model, containing only hidden layers
    intermediate_layer_model = Model(inputs = model.get_layer(input_layer_name).input, outputs = model.get_layer(output_layer_name).output)

    # Extract the latent features in column vector form 
    # from the intermediate layer model
    latent_features = intermediate_layer_model.predict(input_data)

    # Determine the explained variance of each feature in the order they are 
    # presented - they will eventually have to be ordered in terms of variance
    # captured in the original data. This explained variance can be found by
    # determining the covariance matrix of the latent feature matrix where 
    # each row represents one latent feature
    explained_variance_unordered = np.diag(np.cov(latent_features.T))

    # Calculate the total variance captured by each latent feature
    total_variance = sum(explained_variance_unordered)

    # Now we order the explained variance of each feature, but return only the
    # indices. These indices are used to properly order the latent features
    # according to the magnitude of their corresponding explained variance
    explained_variance_ratio_ordered_idx = explained_variance_unordered.argsort()[::-1]  

    # Now order the fraction of explained variance taken up by each latent 
    # feature from highest to lowest
    explained_variance_ratio_ordered = \
      explained_variance_unordered[explained_variance_ratio_ordered_idx] / total_variance

    # Order the latent features according their explained variance from 
    # highest to lowest
    latent_features_ordered = latent_features[:, explained_variance_ratio_ordered_idx]
    
    # Reset model weights after training on the current dataset has been 
    # completed, prior to moving on to the next dataset
    model.reset_states()

################################################################################################
  # If PCA rather than AE is being applied for decomposition, the following 
  # portion of the IF statement will be executed
  else:
    
    # Instantiate PCA class
    pca = PCA()
    
    # Fit PCA instance to the input data
    pca.fit(input_data)

    # Extract the ordered explained variance ratios and latent features 
    # corresponding to those explained variance ratios
    explained_variance_ratio_ordered = pca.explained_variance_ratio_
    latent_features_ordered = np.matmul(input_data, pca.components_)

  # Return explained variance ratios and latent features
  return explained_variance_ratio_ordered, latent_features_ordered

In [ ]:
sharpe_ratio_AR = []
sharpe_ratio_EW = []

ann_ret_AR = []
ann_ret_EW = []

volatility_AR = []
volatility_EW = []

returns_AR_list = []
returns_EQ_list = []


AR_calculation_time = []

ts_absorb_ratio_list = []
ts_components_list = []

#********ADJUSTABLE HYPERPARAMETERS*********************************************

# Select a method of decomposition - either 'PCA' or 'AE'
method = 'AE'

# comp_limit: maximum number of components used to calculate the AR 
# such that (comp_limit < total # of features post-decomposition)
comp_limit = 9

# List of the number of components used to calculate AR in each iteration
comp_iteration = list(range(1, comp_limit + 1))



# List of nodes required for each hidden layer of the AE model - list length
# is the number of hidden layers the model will have. Note that if this list
# is set and the decomposition method is set to 'PCA', it will have no effect
node_count_list = [15, 10, 15]

# Parameter which dictates which strategy for using hidden autoencoder layers 
# is applied to the decomposition process. If "False", all hidden layers will be
# used to decompose the feature space; 
max_encode = False

#*******************************************************************************

for comp in comp_iteration:

  ik = 0
  ewm = True # setting ewm to false means historical returns are equally weighted
  manual_ewm = True  #use manual ewm (instead of pandas ewm function)
  #use_cov_mat = False #use the square shaped covariance of ret_frame instead of t ret_frame directly

  lin_ae = None
  time_start = time.time()
  for ix in range(lookback_window + days_offset, min(num_days, len(asset_returns)), step_size):
      ret_frame = normalize_returns(asset_returns.iloc[ix - lookback_window:ix, :]) # fixed window 
      ret_frame_kmo = ret_frame.copy(deep=True)
      if ewm:

          if manual_ewm:
              #multiply the returns of ret_frame by the ex_probs (hint:transposing is required more than once)
              #hint: make sure smallest weights multiply the oldest returns
              ret_frame = (ret_frame.T * exp_probs).T
    
          else:
              ret_frame = ret_frame.ewm(halflife=252).mean() #for comparison

      
      if ik == 0 or ik % 21 == 0:

          ### fit decomposition method of choice (AE or PCA), compute the number of
          # required latent features/components to meet the variance threshold, 
          # and absorption ratio 
          # store results into components[ik] and absorp_ratio[ik]

          explained_variance, zfeatures = decomposition_function(ret_frame, method = method, node_count_list = node_count_list, max_encode = max_encode)
              
          #Using ret_frame_kmo, calculate the KMO statistic (=kmo_total) and assign it to kmo[ik]. 
          kmo_per_variable, kmo_total = calculate_kmo(ret_frame_kmo)
          kmo[ik] = kmo_total 

          components[ik] = np.sum(np.cumsum(explained_variance) < var_threshold) + 1
          
          #calculate the absorption ratio by calling absorption_ratio() and assign it to absorp_ratio[ik]
          #when you run this loop try various settings for the parameter n_components in this absorption_ratio function.
          absorp_ratio[ik] = absorption_ratio(explained_variance, comp)
 
      else:
          absorp_ratio[ik] = absorp_ratio[ik-1] 
          components[ik] = components[ik-1]
          kmo[ik] = kmo[ik-1]

      ik += 1
  runtime = time.time() - time_start
  print ('Absorption Ratio done! Time elapsed: {} seconds'.format(runtime))   

  AR_calculation_time.append(runtime) 

  ts_components = pd.Series(components, index=components_ts_index)
  ts_absorb_ratio = pd.Series(absorp_ratio, index=components_ts_index)




  #np.amin(kmo)

  # ts_absorb_ratio.plot(figsize=(12,6), title='Absorption Ratio via PCA', linewidth=3)
  # plt.savefig("Absorption_Ratio_20i.png", dpi=900)

  #pd.DataFrame(components).plot()

  # following Kritzman and computing AR_delta = (15d_AR -1yr_AR) / sigma_AR
  ts_ar = ts_absorb_ratio
  ar_mean_1yr = ts_ar.rolling(252).mean()
  ar_mean_15d = ts_ar.rolling(15).mean()
  ar_sd_1yr = ts_ar.rolling(252).std()
  ar_delta = (ar_mean_15d - ar_mean_1yr) / ar_sd_1yr  # standardized shift in absorption ratio

  # df_plot = pd.DataFrame({'AR_delta': ar_delta.values, 'AR_1yr': ar_mean_1yr.values, 'AR_15d': ar_mean_15d.values}, 
  #                        index=ts_ar.index)
  # df_plot = df_plot.dropna()
  # if df_plot.shape[0] > 0:
  #     df_plot.plot(figsize=(12, 6), title='Absorption Ratio Delta', linewidth=3, secondary_y=['AR_1yr','AR_15d'])

  #Average trades per year
  ar_delta_data = ar_delta[251:]

  rebal_dates = np.zeros(len(ar_delta_data))
  wgts = pd.DataFrame(data=np.zeros((len(ar_delta_data.index), 2)), index=ar_delta_data.index, columns=('EQ', 'FI'))

  prtf_wgts = get_weight(ar_delta_data.values[0])
  wgts.iloc[0, :] = prtf_wgts
  for ix in range(1, len(ar_delta_data)):
      prtf_wgts = get_weight(ar_delta_data.values[ix])
      wgts.iloc[ix, :] = prtf_wgts
      if wgts.iloc[ix-1, :][0] != prtf_wgts[0]:
          prtf_wgts = wgts.iloc[ix, :]
          rebal_dates[ix] = 1

  ts_rebal_dates = pd.Series(rebal_dates, index=ar_delta_data.index)
  ts_trades_per_year = ts_rebal_dates.groupby([ts_rebal_dates.index.year]).sum()
  print('Average number of trades per year %.2f' % ts_trades_per_year.mean())
   



  ann_ret, ann_vol, sharpe, returns_AR = backtest_strategy(wgts, etf_returns)
  print('Absorption Ratio strategy:', ann_ret, ann_vol, sharpe)

  returns_AR_list.append(returns_AR)

  ts_components_list.append(ts_components.copy())
  ts_absorb_ratio_list.append(ts_absorb_ratio.copy())

  ann_ret_AR.append(ann_ret)
  volatility_AR.append(ann_vol)
  sharpe_ratio_AR.append(sharpe)

  eq_wgts = wgts.copy()
  eq_wgts.iloc[:, ] = 0.5
  ann_ret_eq_wgt, ann_vol_eq_wgt, sharpe_eq_wgt, returns_EQ = backtest_strategy(eq_wgts, etf_returns)
  print('Equally weighted:', ann_ret_eq_wgt, ann_vol_eq_wgt, sharpe_eq_wgt)

  ann_ret_EW.append(ann_ret_eq_wgt)
  volatility_EW.append(ann_vol_eq_wgt)
  sharpe_ratio_EW.append(sharpe_eq_wgt)

  returns_EQ_list.append(returns_EQ)


for x in range(0, len(returns_AR_list)):
  print(str(comp_iteration[x]) + ' Components')
  print('Absorption Ratio strategy:', ann_ret_AR[x], volatility_AR[x], sharpe_ratio_AR[x])
  print('Equally weighted:', ann_ret_EW[x], volatility_EW[x], sharpe_ratio_EW[x])

  
  plt.title('Cumulative Portfolio Return for AR and EW Strategies')
  plt.plot(returns_AR_list[x], label = 'Cumulative Portfolio Return ' + str(comp_iteration[x]) + ' Component AR Strategy')
  plt.plot(returns_EQ_list[x], label = 'Cumulative Portfolio Return EW Strategy')
  plt.xticks(rotation = 45)
  plt.ylabel('Cumulative Portfolio Return')
  plt.xlabel('Date')
  plt.legend(loc = (1.05, 0.85))
  plt.show()


for x in range(0, len(ts_components_list)):
  print(str(comp_iteration[x]) + ' Components')
  pd.DataFrame(ts_components_list[x]).plot()
  plt.show()

for x in range(0, len(ts_absorb_ratio_list)):
  print(str(comp_iteration[x]) + ' Components')
  ts_absorb_ratio_list[x].plot(figsize=(12,6), title='Absorption Ratio via ' + method + ' (' + str(comp_iteration[x]) + ' Components)', linewidth=3)
  plt.show()



for x in range(0, len(ts_absorb_ratio_list)):
  print(str(comp_iteration[x]) + ' Components')

    
  # following Kritzman and computing AR_delta = (15d_AR -1yr_AR) / sigma_AR
  ts_ar = ts_absorb_ratio_list[x]
  ar_mean_1yr = ts_ar.rolling(252).mean()
  ar_mean_15d = ts_ar.rolling(15).mean()
  ar_sd_1yr = ts_ar.rolling(252).std()
  ar_delta = (ar_mean_15d - ar_mean_1yr) / ar_sd_1yr  # standardized shift in absorption ratio

  df_plot = pd.DataFrame({'AR_delta': ar_delta.values, 'AR_1yr': ar_mean_1yr.values, 'AR_15d': ar_mean_15d.values}, 
                        index=ts_ar.index)
  df_plot = df_plot.dropna()
  if df_plot.shape[0] > 0:
      df_plot.plot(figsize=(12, 6), title='Absorption Ratio Delta', linewidth=3, secondary_y=['AR_1yr','AR_15d'])
      plt.show()



  #NOTE: THE GRAPH AND RESULTS BELOW ARE ONLY APPROXIMATE

In [ ]:
# Function to plot a metric as a function of number of decomposed 
# components used to calculate AR
def metric_trend(metric_series, metric_name):

  component_list = np.arange(1,len(metric_series) + 1)
  plt.plot(component_list, metric_series)
  plt.xticks(component_list)
  if metric_name != 'AR Calculation Time':
    plt.title(metric_name + ' vs. Number of Components (AR Strategy)')
    plt.ylabel(metric_name)
  else:
    plt.title(metric_name + ' vs. Number of Components')   
    plt.ylabel(metric_name + ' (s)')
  plt.xlabel('Number of Components')
  plt.show()

In [ ]:
# Function to plot a metric as a function of number of decomposed 
# components used to calculate AR to develop a trading strategy, compared to
# the metric achieved by a baseline trading strategy - in our case, equally
# weighted FI and EQ
def metric_trend_compare(metric_series, metric_series_baseline, metric_name):

  component_list = np.arange(1,len(metric_series) + 1)
  plt.plot(component_list, metric_series, label = 'AR Strategy')
  plt.plot(component_list, metric_series_baseline, label = 'EW Strategy')
  plt.xticks(component_list)
  plt.title(metric_name + ' vs. Number of Components')
  plt.xlabel('Number of Components')
  plt.ylabel(metric_name)

  plt.legend(loc = (1.05,0.85))
  plt.show()

In [ ]:
# Print metric results for different component counts
metric_trend(sharpe_ratio_AR, 'Annualized Sharpe Ratio')
metric_trend(volatility_AR, 'Annualized Volatility')
metric_trend(ann_ret_AR, 'Annualized Return')
metric_trend(AR_calculation_time, 'AR Calculation Time')
metric_trend_compare(sharpe_ratio_AR, sharpe_ratio_EW, 'Annualized Sharpe Ratio')
metric_trend_compare(volatility_AR, volatility_EW, 'Annualized Volatility')
metric_trend_compare(ann_ret_AR, ann_ret_EW, 'Annualized Return')

In [ ]:
# Tabulation of metric results
df = pd.DataFrame([sharpe_ratio_AR, volatility_AR, ann_ret_AR, AR_calculation_time], ).T
df.index = np.arange(1,comp_limit + 1)
df.columns = ['SHARPE', 'VOL', 'ANN_RET', 'AR_CALC_TIME']
df.sort_values(by = 'SHARPE', ascending = False)